In [1]:
using CSV
using DataFrames
using Statistics
using ProgressBars
using LinearAlgebra
using BenchmarkTools
using TimerOutputs

In [2]:
ROOT_DIR = "santa-2022"
IMAGE_FILE = "$ROOT_DIR/image.csv"

"santa-2022/image.csv"

In [3]:
df = DataFrame(CSV.File(IMAGE_FILE, typemap=Dict(Int64=>Int16, Float64=>Float32)))
first(df)

Row,x,y,r,g,b
,Int16,Int16,Float32,Float32,Float32
1,-128,128,0.6,0.619608,0.654902


In [4]:
function cartesian_to_array(x, y, shape)::Vector{Int16}
    m, n = shape[1:2]
    i = (n - 1) ÷ 2 - y
    j = (n - 1) ÷ 2 + x
    
    if i < 0 || i >= m || j < 0 || j >= n
        throw(ErrorException("Coordinates not within given dimensions."))
    end
    return [i, j]
end

cartesian_to_array (generic function with 1 method)

In [5]:
function array_to_cartesian(i, j, shape)
    m, n = shape[1:2]
    
    if i < 0 || i >= m || j < 0 || j >= n
        throw(ErrorException("Coordinates not within given dimensions."))
    end
    
    y = (n - 1) ÷ 2 - i
    x = j - (n - 1) ÷ 2
    return x, y
end

array_to_cartesian (generic function with 1 method)

In [6]:
point = [1, 8]
shape = [9, 9, 3]

3-element Vector{Int64}:
 9
 9
 3

In [7]:
array_to_cartesian((1, 8)..., shape)

(4, 3)

In [8]:
cartesian_to_array(array_to_cartesian(point..., shape)..., shape)

2-element Vector{Int16}:
 1
 8

In [9]:
cartesian_to_array(array_to_cartesian(point..., shape)..., shape) == point

true

In [10]:
function df_to_image(df)
    d = Dict{Tuple{Int16, Int16}, Vector{Float32}}()
    for r in eachrow(df)
        x, y = r.x, r.y
        v = r[["r", "g", "b"]] |> collect
        d[x,y] = v
    end
    
    return d
end

df_to_image (generic function with 1 method)

In [11]:
image = df_to_image(df)

Dict{Tuple{Int16, Int16}, Vector{Float32}} with 66049 entries:
  (-125, -62) => [0.552941, 0.686275, 0.92549]
  (-112, 116) => [0.854902, 0.901961, 0.980392]
  (-81, 79)   => [0.913725, 0.490196, 0.34902]
  (19, -90)   => [0.294118, 0.572549, 0.298039]
  (98, -42)   => [0.584314, 0.709804, 0.929412]
  (7, -27)    => [0.992157, 0.94902, 0.666667]
  (48, 61)    => [0.756863, 0.835294, 0.964706]
  (-77, -31)  => [0.858824, 0.858824, 0.854902]
  (-62, 37)   => [0.356863, 0.352941, 0.34902]
  (52, 32)    => [0.784314, 0.85098, 0.952941]
  (25, 73)    => [0.780392, 0.85098, 0.968627]
  (91, 30)    => [0.705882, 0.796078, 0.952941]
  (100, 10)   => [0.670588, 0.772549, 0.94902]
  (-4, 60)    => [0.423529, 0.752941, 0.4]
  (-97, -69)  => [0.533333, 0.678431, 0.92549]
  (-2, 38)    => [0.211765, 0.458824, 0.215686]
  (-35, 15)   => [0.341176, 0.686275, 0.345098]
  (-92, 67)   => [0.898039, 0.482353, 0.337255]
  (-44, 77)   => [0.788235, 0.854902, 0.968627]
  (70, -16)   => [0.631373, 0.741176, 

In [12]:
println("Min x: ", min(df.x...))
println("Min x: ", max(df.x...))
println("Min y: ", min(df.y...))
println("Min y: ", max(df.y...))

Min x: -128
Min x: 128
Min y: -128
Min y: 128


In [13]:
function get_position(config)
    return reduce((p, q) -> (p[1] + q[1], p[2] + q[2]), config; init= (0, 0))
end

config = [[4, 0], [-2, 2], [-1, 0], [-1, 1]]

get_position(config)

(0, 3)

In [14]:
function rotate_link(vector, direction)
    x, y = vector
    if direction == 1  # counter-clockwise
        if y >= x && y > -x
            x -= 1
        elseif y > x && y <= -x
            y -= 1
        elseif y <= x && y < -x
            x += 1
        else
            y += 1
        end
    elseif direction == -1  # clockwise
        if y > x && y >= -x
            x += 1
        elseif y >= x && y < -x
            y += 1
        elseif y < x && y <= -x
            x -= 1
        else
            y -= 1
        end
    end
    return [x, y]
end


function rotate(config, i, direction)
    new_config = deepcopy(config)
    new_config[i] = rotate_link(config[i], direction)
    return new_config
end


function get_square(link_length)
    link = (link_length, 0)
    coords = [link]
    for _ in 1:(8 * link_length - 1)
        link = rotate_link(link, direction=1)
        push!(coords, link)
    end
    return coords
end

get_square (generic function with 1 method)

In [15]:
for _ in 1:16
    println(config)
    config = rotate(config, 1, 1)
end

[[4, 0], [-2, 2], [-1, 0], [-1, 1]]
[[4, 1], [-2, 2], [-1, 0], [-1, 1]]
[[4, 2], [-2, 2], [-1, 0], [-1, 1]]
[[4, 3], [-2, 2], [-1, 0], [-1, 1]]
[[4, 4], [-2, 2], [-1, 0], [-1, 1]]
[[3, 4], [-2, 2], [-1, 0], [-1, 1]]
[[2, 4], [-2, 2], [-1, 0], [-1, 1]]
[[1, 4], [-2, 2], [-1, 0], [-1, 1]]
[[0, 4], [-2, 2], [-1, 0], [-1, 1]]
[[-1, 4], [-2, 2], [-1, 0], [-1, 1]]
[[-2, 4], [-2, 2], [-1, 0], [-1, 1]]
[[-3, 4], [-2, 2], [-1, 0], [-1, 1]]
[[-4, 4], [-2, 2], [-1, 0], [-1, 1]]
[[-4, 3], [-2, 2], [-1, 0], [-1, 1]]
[[-4, 2], [-2, 2], [-1, 0], [-1, 1]]
[[-4, 1], [-2, 2], [-1, 0], [-1, 1]]


In [16]:
config = [[4, 0], [-2, 2], [-1, 0], [-1, 1]]

4-element Vector{Vector{Int64}}:
 [4, 0]
 [-2, 2]
 [-1, 0]
 [-1, 1]

In [17]:
function get_neighbors(config)::Vector{Vector{Vector{Int16}}}
    directions = Iterators.product([[-1, 0, 1] for _ in 1:length(config)]...) |> collect |> vec
    list_directions = [enumerate(d) |> collect for d in directions]
    
    list_rotate = [reduce((x, y) -> rotate(x, y...), d; init=config) for d in list_directions]
    filter!(e -> e != config, list_rotate)
    
    return list_rotate
end

get_neighbors (generic function with 1 method)

In [18]:
get_neighbors(config)

80-element Vector{Vector{Vector{Int16}}}:
 [[4, -1], [-1, 2], [-1, 1], [0, 1]]
 [[4, 0], [-1, 2], [-1, 1], [0, 1]]
 [[4, 1], [-1, 2], [-1, 1], [0, 1]]
 [[4, -1], [-2, 2], [-1, 1], [0, 1]]
 [[4, 0], [-2, 2], [-1, 1], [0, 1]]
 [[4, 1], [-2, 2], [-1, 1], [0, 1]]
 [[4, -1], [-2, 1], [-1, 1], [0, 1]]
 [[4, 0], [-2, 1], [-1, 1], [0, 1]]
 [[4, 1], [-2, 1], [-1, 1], [0, 1]]
 [[4, -1], [-1, 2], [-1, 0], [0, 1]]
 [[4, 0], [-1, 2], [-1, 0], [0, 1]]
 [[4, 1], [-1, 2], [-1, 0], [0, 1]]
 [[4, -1], [-2, 2], [-1, 0], [0, 1]]
 ⋮
 [[4, -1], [-2, 1], [-1, 0], [-1, 0]]
 [[4, 0], [-2, 1], [-1, 0], [-1, 0]]
 [[4, 1], [-2, 1], [-1, 0], [-1, 0]]
 [[4, -1], [-1, 2], [-1, -1], [-1, 0]]
 [[4, 0], [-1, 2], [-1, -1], [-1, 0]]
 [[4, 1], [-1, 2], [-1, -1], [-1, 0]]
 [[4, -1], [-2, 2], [-1, -1], [-1, 0]]
 [[4, 0], [-2, 2], [-1, -1], [-1, 0]]
 [[4, 1], [-2, 2], [-1, -1], [-1, 0]]
 [[4, -1], [-2, 1], [-1, -1], [-1, 0]]
 [[4, 0], [-2, 1], [-1, -1], [-1, 0]]
 [[4, 1], [-2, 1], [-1, -1], [-1, 0]]

In [19]:
function reconfiguration_cost(from_config, to_config)
    nlinks = length(from_config)
    mat_from = hcat(from_config...)
    mat_to = hcat(to_config...)
    abs_diff = abs.(mat_from - mat_to)
    
    return sqrt(sum(abs_diff))
end

function color_cost(from_position, to_position, image, color_scale=3.0)
    return sum(abs.(image[from_position] - image[to_position])) * color_scale
end

function step_cost(from_config, to_config, image)
    from_position = get_position(from_config)
    to_position = get_position(to_config)
    
    return reconfiguration_cost(from_config, to_config) + color_cost(from_position, to_position, image)
end

function total_cost(path, image)
    return reduce(
        (cost, pair) -> cost + step_cost(pair[begin], pair[end], image),
        zip(path[begin:end-1], path[begin+1:end]) |> collect;
        init=0,
    )
end

total_cost (generic function with 1 method)

In [20]:
function get_direction(u, v)
    direction = sign(LinearAlgebra.cross([u..., 0], [v...,0])[end])
    
    if direction==0 && LinearAlgebra.dot(u, v) < 0
        direction=1
    end
    
    return direction
end

function get_angle(u, v)
    cross_prod = LinearAlgebra.cross([u..., 0], [v...,0])[end]
    dot_prod = LinearAlgebra.dot(u, v)
    return rad2deg(atan(cross_prod, dot_prod))
end 

get_angle (generic function with 1 method)

In [21]:
function get_path_to_point(config, point)
    path = [config]
    
    
    for i in 1:length(config)
        link = config[i]
        base = begin 
            if i==1
                get_position([])
            else
                get_position(config[begin:(i-1)])
            end
        end
        relbase = [point[1] - base[1], point[2] - base[2]]
        position = get_position(config[begin:i])
        relpos = [point[1] - position[1], point[2] - position[2]]

        radius = reduce((r, link) -> r + max(abs.(link)...), config[i+1:end]; init=0)
        
        if radius==1 && relpos==[0,0]
            config = rotate(config, i, 1)
            if get_position(config) == Tuple(point)
                push!(path, config)
                break
            else
                continue
            end
        end
        
    
            
        while max(abs.(relpos)...) > radius
            direction = get_direction(link, relbase)
            config = rotate(config, i, direction)
            push!(path, config)
            link = config[i]
            base = begin 
                if i==1
                    get_position([])
                else
                    get_position(config[begin:(i-1)])
                end
            end
            
            relbase = [point[1] - base[1], point[2] - base[2]]
            position = get_position(config[begin:i])
            relpos = [point[1] - position[1], point[2] - position[2]]

            radius = reduce((r, link) -> r + max(abs.(link)...), config[i+1:end]; init=0)
        end
    end
    
    if get_position(path[end]) != Tuple(point)
        throw(ErrorException("Something went wrong!"))
    end
    return path
end

get_path_to_point (generic function with 1 method)

In [22]:
function get_path_to_configuration(from_config, to_config)
    path = [from_config]
    config = deepcopy(from_config)
    while config != to_config
        for i in 1:length(config)
            config = rotate(config, i, get_direction(config[i], to_config[i]))
        end
        push!(path, config)
    end
    
    if path[end] != to_config
        throw(ErrorException("Something went wrong!"))
    end

    return path
end

get_path_to_configuration (generic function with 1 method)

In [23]:
function quick_start_solution()
    origin = [[64, 0], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
    n = origin[begin][begin] * 2

    points = Iterators.product(-n:n |> collect, -n:n |> collect) |> collect |> vec
    sort!(points) # total_cost 166305 in Python :D
    
    path = [origin]
    for p in ProgressBar(points)
        config = path[end]
        append!(path, get_path_to_point(config, p)[2:end])
    end

    append!(path, get_path_to_configuration(path[end], origin)[2:end])
    
    return path
end

quick_start_solution (generic function with 1 method)

In [24]:
@time path=quick_start_solution()

0.0%┣                                           ┫ 0/66.0k [00:00<-55:-3, -0s/it]
0.0%┣                                        ┫ 1/66.0k [00:00<Inf:Inf, InfGs/it]
7.1%┣██▊                                   ┫ 4.7k/66.0k [00:00<00:03, 19.7kit/s]
15.6%┣█████▋                              ┫ 10.3k/66.0k [00:00<00:02, 34.8kit/s]
25.3%┣█████████▏                          ┫ 16.7k/66.0k [00:00<00:01, 48.4kit/s]
32.2%┣███████████▋                        ┫ 21.2k/66.0k [00:00<00:01, 53.7kit/s]
38.7%┣██████████████                      ┫ 25.6k/66.0k [00:00<00:01, 57.3kit/s]
48.6%┣█████████████████▌                  ┫ 32.1k/66.0k [00:00<00:01, 64.6kit/s]
53.6%┣███████████████████▎                ┫ 35.4k/66.0k [00:01<00:00, 64.7kit/s]
63.0%┣██████████████████████▊             ┫ 41.6k/66.0k [00:01<00:00, 69.7kit/s]
67.3%┣████████████████████████▎           ┫ 44.4k/66.0k [00:01<00:00, 68.6kit/s]
77.0%┣███████████████████████████▊        ┫ 50.8k/66.0k [00:01<00:00, 72.8kit/s]
77.8%┣██████████████████████

  0.956044 seconds (10.39 M allocations: 763.603 MiB, 28.33% gc time, 15.34% compilation time)


91.2%┣████████████████████████████████▉   ┫ 60.2k/66.0k [00:01<00:00, 63.8kit/s]
100.0%┣███████████████████████████████████┫ 66.0k/66.0k [00:01<00:00, 66.4kit/s]


133604-element Vector{Vector{Vector{Int64}}}:
 [[64, 0], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -1], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -2], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -3], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -4], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -5], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -6], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -7], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -8], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -9], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -10], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -11], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]
 [[64, -12], [-32, 0], [-16, 

In [25]:
total_cost(path, image)

166305.28443298

In [26]:
function config_to_string(config)
    return join([join(v, ' ') for v in config], ';')
end

config_to_string (generic function with 1 method)

In [27]:
config_to_string(path[begin])

"64 0;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0"

In [28]:
submission = DataFrame((configuration=[config_to_string(p) for p in path]))

Row,configuration
,String
1,64 0;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
2,64 -1;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
3,64 -2;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
4,64 -3;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
5,64 -4;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
6,64 -5;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
7,64 -6;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
8,64 -7;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
9,64 -8;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0


In [29]:
CSV.write("submission.csv", submission) 

"submission.csv"